In [1]:
import urllib2 as ul
from bs4 import BeautifulSoup as bs
import time
import re
import codecs

In [2]:
def HanParse(pageno,write=0):
    newspage = ul.urlopen('http://baduk.hangame.com/news.nhn?gseq='+str(pageno)+'&m=view&page=1&searchfield=&leagueseq=0&searchtext=')
    soup = bs(newspage,'html.parser')
    #[s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    tmp = soup.getText()
    tmp=u'내용'.join(tmp.split(u'내용')[2:])
    b=tmp.find(u'관련 뉴스보기')
    tmp=tmp[:b]
    tmp='\n'.join([i for i in tmp.split('\n') if len(i)>0])
    if write==1:
        h=codecs.open('./han/'+str(pageno)+'.d','w',encoding='utf-8')
        h.write(tmp)
    return tmp

def TyParse(pageno,write=0):
    newspage = ul.urlopen('http://www.tygem.com/news/news/viewpage.asp?seq='+str(pageno)+'&gubun=&igubun=&find=&findword=')
    soup = bs(newspage,'html.parser')
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    tmp = soup.getText()
    a=tmp.find(u'Home > 뉴스        Daily뉴스')
    b=tmp.find(u'TYGEM / ')
    tmp = [i for i in tmp[a:b].split('\n') if len(i)>0]
    vtext = '\n'.join(tmp[1:])
    if write==1:
        h=codecs.open('./dat/'+str(pageno)+'.d','w',encoding='utf-8')
        h.write(vtext)
    return vtext

def OroParse(pageno,write=0):
    newspage = ul.urlopen('http://www.cyberoro.com/news/news_view.oro?div_no=A1&num='+str(pageno)+'&pageNo=1&cmt_n=0')
    soup = bs(newspage,'html.parser')
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    tmp = soup.getText()
    a=tmp.find(u'Home > 뉴스')
    b=tmp.find(u'┃관련뉴스')
    tmp = tmp[a:b]
    tmp = [i for i in tmp.split('\n')[1:] if len(i)>0]
    vis = '\n'.join(tmp)
    if write==1:
        h=codecs.open('./oro/'+str(pageno)+'.d','w',encoding='utf-8')
        h.write(vis)
    return vis

def GetSent(pageno):
    dat=HanParse(pageno)
    sent = dat.split('.')[6:-1]
    first = sent[0].split(u'내용')
    sent[0]=first[-1]
    return sent[:-3]

def ReadSent(pageno,src='./han/'):
    dat=codecs.open(src+str(pageno)+'.d',encoding='utf-8')
    dat = dat.read()
    dat = Clean(dat)
    dat = dat.replace('?','.')
    dat = dat.replace('!','.')
    return [' '.join(i.split()) for i in dat.split('.')]
    
def Clean(sentence,comma=0):
    if comma==0:
        for i in ['\n','_','-','(',')','"','\'',u'▲','...','[',']',u'■','<','>','\r']:
            sentence=sentence.replace(i,' ')
    else:
        for i in ['\n','_','-','(',')','"','\'',u'▲','...','[',']',u'■','<','>',',','\r']:
            sentence=sentence.replace(i,' ')
    return sentence.strip()

def GetHanArticles(indexno):
    articles=[]
    main=ul.urlopen('http://baduk.hangame.com/news.nhn?&page='+str(indexno))
    mainbs=bs(main,'html.parser')
    tmp = [link.get('href') for link in mainbs.find_all('a')]
    for j in tmp:
        try:
            if j.find('readnews')>0:
                articles.append(j)
        except AttributeError:
            continue
    mynews=[int(re.search(r'\d+', i).group()) for i in articles]
    mynews = sorted(list(set(mynews)))
    return mynews

def GetTyArticles(indexno):
    articles=[]
    main=ul.urlopen('http://www.tygem.com/news/news/list.asp?pagec='+str(indexno))
    mainbs=bs(main,'html.parser')
    tmp = [link.get('href') for link in mainbs.find_all('a')]
    for j in tmp:
        try:
            if j.find('/news/news/view.asp?gubun=&seq=')>-1:
                articles.append(j)
        except AttributeError:
            continue
    mynews=[int(re.search(r'\d+', i).group()) for i in articles]
    mynews = sorted(list(set(mynews)))
    return mynews

def GetOroArticles(pageno):
    articles=[]
    loc="http://cyberoro.com/news/news_list.oro?cmt_n=0&div_no=A1&pageNo="+str(pageno)+"&blockNo=1"
    newspage = ul.urlopen(loc)
    soup = bs(newspage,'html.parser')
    tmp = [link.get('href') for link in soup.find_all('a')]
    for j in tmp:
        try:
            if j.find('news_view.oro?pageNo=')>0:
                articles.append(j)
        except AttributeError:
            continue
    articles = [i[i.find('num'):] for i in articles]
    mynews=[int(re.search(r'\d+', i).group()) for i in articles]
    return mynews

def WordCount(pagelist,sort=1,sentence=0):
    sentences=[]
    words=[]
    for page in pagelist:
        
        tmp=GetSent(page)
        for sent in tmp:
            sentences.append(Clean(sent))
    for sent in sentences:
        for j in sent.split():
            words.append(j)
    uwords = list(set(words))
    wc = [(i,words.count(i)) for i in uwords]
    if sort==1:
        wc = sorted(wc, key=lambda x: x[1])
        wc.reverse()
    if sentence==0:
        return wc
    else:
        return wc, sentence

#a la https://github.com/mouuff/Google-Translate-API/blob/master/python/GoogleTranslate.py
def translate(to_translate, to_langage="auto", langage="auto"):
    to_translate=to_translate.encode('utf-8')
    agents = {'User-Agent':"Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1;SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727; .NET CLR 3.0.04506.30)"}
    before_trans = 'class="t0">'
    link = "http://translate.google.com/m?hl=%s&sl=%s&q=%s"% (to_langage, langage, to_translate.replace(" ", "+"))
    request = ul.Request(link, headers=agents)
    page = ul.urlopen(request).read()
    result = page[page.find(before_trans)+len(before_trans):]
    result = result.split("<")[0]
    return result

def UpdateData(src='han'):
    if src=='han':
        f=open('./lists/han.list')
        files = [int(line.split('.')[0]) for line in f]
        cnt=1
        articles=GetHanArticles(1)
        while min(articles) not in files:
            cnt+=1
            articles+=GetHanArticles(cnt)
        articles=[j for j in articles if j not in files]
        print 'updating ', len(articles), ' hangame files'    
        for i in articles:HanParse(i,write=1)
        return articles
    
    if src=='ty':
        f=open('./lists/dat.list')
        files = [int(line.split('.')[0]) for line in f]
        cnt=1
        articles=GetTyArticles(1)
        while min(articles) not in files:
            cnt+=1
            articles+=GetTyArticles(cnt)
        articles=[j for j in articles if j not in files]
        print 'updating ', len(articles), ' tygem files'    
        for i in articles:TyParse(i,write=1)
        return articles

    if src=='oro':
        f=open('./lists/oro.list')
        files = [int(line.split('.')[0]) for line in f]
        articles=GetOroArticles(1)
        cnt=1
        checks = [i in files for i in articles]
        while checks.count(True)==0 :
            cnt+=1
            articles+=GetOroArticles(cnt)
            checks = [i in files for i in articles]
            print cnt
        articles=[j for j in articles if j not in files]
        print 'updating ', len(articles), ' oro files'    
        for i in articles:OroParse(i,write=1)
        return articles
    
    if src=='all':
        articles=UpdateData(src='han')
        articles+=UpdateData(src='ty')
        articles+=UpdateData(src='oro')
        return articles

def MergeWords():
    pass

In [4]:
UpdateData(src='ty')


updating  1216  tygem files


KeyboardInterrupt: 

In [5]:
UpdateData(src='oro')

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
updating  1288  oro files


No handlers could be found for logger "bs4.dammit"


[523671,
 523672,
 523670,
 523669,
 523668,
 523667,
 523666,
 523665,
 523663,
 523662,
 523660,
 523661,
 523659,
 523655,
 523658,
 523657,
 523653,
 523651,
 523652,
 523650,
 523648,
 523646,
 523647,
 523645,
 523644,
 523643,
 523641,
 523639,
 523635,
 523638,
 523636,
 523634,
 523633,
 523631,
 523630,
 523629,
 523628,
 523627,
 523625,
 523624,
 523623,
 523622,
 523621,
 523619,
 523618,
 523620,
 523617,
 523615,
 523614,
 523613,
 523612,
 523611,
 523610,
 523609,
 523608,
 523598,
 523607,
 523606,
 523604,
 523603,
 523602,
 523601,
 523600,
 523599,
 523595,
 523594,
 523593,
 523587,
 523591,
 523592,
 523590,
 523589,
 523588,
 523586,
 523585,
 523583,
 523582,
 523581,
 523580,
 523579,
 523578,
 523577,
 523572,
 523576,
 523575,
 523573,
 523569,
 523568,
 523566,
 523564,
 523565,
 523563,
 523562,
 523561,
 523560,
 523559,
 523558,
 523557,
 523556,
 523555,
 523553,
 523554,
 523552,
 523551,
 523550,
 523549,
 523548,
 523543,
 523533,
 523547,
 523546,
 